In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
from dotenv import load_dotenv
import os

import warnings
warnings.filterwarnings("ignore")

load_dotenv()
directory = os.getenv("path")

In [ ]:
def ChangeExemplarsToBinary(datasetPath: str) -> pd.DataFrame:
    toBinary_df = pd.read_csv(datasetPath)
    toBinary_df = toBinary_df.sort_values(by=["station_id"])
    toBinary_df = toBinary_df.reset_index(drop=True)
    toBinary_df.index += 1
    toBinary_df = toBinary_df.iloc[:, 1:]

    for col in toBinary_df.columns:
        unique_values = toBinary_df[col].unique().tolist()
        toBinary_df[col] = toBinary_df.index.map(
            lambda x: 1 if x in unique_values else 0
        )
        # for each column name add _ classification to the name
        # toBinary_df.rename(columns={col: col + "_" + datasetPath}, inplace=True)

    return toBinary_df

# test:
datasetPath = os.path.join(directory, "computed_data/ap_results", "func.csv")
ChangeExemplarsToBinary(datasetPath)

In [9]:
def exemplarStabilityIndex(classification):
    df = ChangeExemplarsToBinary(classification)
    nameClass = classification.split("-")[1]
    df[f"sum_exemplars_{nameClass}"] = df.sum(axis=1)
    sum_list = df[f"sum_exemplars_{nameClass}"].to_list()
    return sum_list

In [ ]:
func_df = exemplarStabilityIndex(os.path.join(directory, "computed_data/ap_results", "func.csv"))
func_climate_df = exemplarStabilityIndex(os.path.join(directory, "computed_data/ap_results", "func_climate.csv"))
func_climate_struct_df = exemplarStabilityIndex(os.path.join(directory, "computed_data/ap_results", "all_attributes.csv"))

# df = pd.DataFrame({"F": func_df, "FC": func_climate_df, "FCS": func_climate_struct_df})
# df["All"] = df.sum(axis=1)
# df

func_df